In [27]:
import pandas as pd
import datetime as dt
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [28]:
df = pd.read_csv('historical_spy_trades_agg.csv')
df

,period,movement,chg,vpoc,total_volume,vpoc_price,tpoc_price,open,high,low,...,price_skew,price_kurt,vol_price_cv,vol_price_skew,vol_price_kurt,trapped_longs,trapped_shorts,trapped_shorts_at_extremes,trapped_longs_at_extremes,trapped_at_extremes_volume
0,2024-04-22 09:30:00,0,-0.1265,0.247937,628143.0,497.88,497.74,497.84,498.07,497.17,...,-0.342,-0.787,2.370,8.560,78.235,1,0,0,0,0.000000
1,2024-04-22 09:35:00,1,0.0080,0.085825,206035.0,497.23,497.24,497.20,497.33,496.92,...,-0.335,-0.872,0.881,1.170,1.209,0,1,0,0,0.000000
2,2024-04-22 09:40:00,1,0.0080,0.075736,166394.0,497.11,497.00,497.24,497.34,496.74,...,-0.027,-0.635,1.006,1.646,2.898,0,1,1,0,0.905946
3,2024-04-22 09:45:00,1,0.1287,0.036437,159041.0,497.43,497.60,497.29,497.96,497.14,...,0.199,-0.706,0.777,0.935,0.057,0,1,0,0,0.000000
4,2024-04-22 09:50:00,0,-0.0301,0.082666,129267.0,498.00,497.86,497.94,498.01,497.68,...,0.304,-0.391,0.806,0.777,-0.466,1,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,2024-05-03 15:35:00,1,0.0704,0.066151,148962.0,511.57,511.56,511.22,511.75,511.14,...,-0.976,-0.134,0.994,1.220,0.818,0,1,0,0,0.000000
776,2024-05-03 15:40:00,0,-0.0137,0.110382,90649.0,511.51,511.49,511.58,511.74,511.40,...,0.775,-0.221,0.948,1.551,2.396,0,0,1,0,0.575009
777,2024-05-03 15:45:00,0,-0.0665,0.089017,187493.0,511.50,511.50,511.51,511.57,511.16,...,-0.690,-0.326,0.935,1.016,0.284,1,0,0,0,0.000000
778,2024-05-03 15:50:00,1,0.0685,0.068252,341953.0,511.40,511.40,511.17,511.72,511.17,...,0.020,-0.730,0.859,1.252,1.373,0,1,0,0,0.000000


In [29]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt

df1 = pd.read_csv('historical_spy_trades_agg.csv')

df1 = df1.set_index("period")
df1.index = pd.to_datetime(df1.index)
df1 = df1.loc["2024-05-03 09:30:00":"2024-05-03 11:00:00"]

layout = go.Layout(
    autosize=False,
    width=2000,
    height=1000,
)

fig = go.Figure(data=[go.Candlestick(x=df1.index,
                open=df1['open'],
                high=df1['high'],
                low=df1['low'],
                close=df1['close'],
                hovertext=df1[['trapped_shorts_at_extremes','trapped_longs_at_extremes']]
                ),
                go.Scatter(x=df1.index, y=df1.vpoc_price, mode="markers",marker=dict(size=10, symbol="arrow-right", color='yellow')),
                go.Scatter(x=df1.index, y=df1.tpoc_price, mode="markers",marker=dict(size=10, symbol="arrow-right", color='violet'))
                ],layout=layout)
fig.show()

In [30]:
df.iloc[:,3:] = df.iloc[:,3:].shift(1,fill_value=-1)
df = df.rename(columns={c: c+"_prev_period" for c in df.columns if c not in ['period','movement','chg']})
df.insert(2,'movement_prev_period',df.movement.shift(1, fill_value=-1))
df.insert(4,'chg_prev_period',df.chg.shift(1))
df

,period,movement,movement_prev_period,chg,chg_prev_period,vpoc_prev_period,total_volume_prev_period,vpoc_price_prev_period,tpoc_price_prev_period,open_prev_period,...,price_skew_prev_period,price_kurt_prev_period,vol_price_cv_prev_period,vol_price_skew_prev_period,vol_price_kurt_prev_period,trapped_longs_prev_period,trapped_shorts_prev_period,trapped_shorts_at_extremes_prev_period,trapped_longs_at_extremes_prev_period,trapped_at_extremes_volume_prev_period
0,2024-04-22 09:30:00,0,-1,-0.1265,NaN,-1.000000,-1.0,-1.00,-1.00,-1.00,...,-1.000,-1.000,-1.000,-1.000,-1.000,-1,-1,-1,-1,-1.000000
1,2024-04-22 09:35:00,1,0,0.0080,-0.1265,0.247937,628143.0,497.88,497.74,497.84,...,-0.342,-0.787,2.370,8.560,78.235,1,0,0,0,0.000000
2,2024-04-22 09:40:00,1,1,0.0080,0.0080,0.085825,206035.0,497.23,497.24,497.20,...,-0.335,-0.872,0.881,1.170,1.209,0,1,0,0,0.000000
3,2024-04-22 09:45:00,1,1,0.1287,0.0080,0.075736,166394.0,497.11,497.00,497.24,...,-0.027,-0.635,1.006,1.646,2.898,0,1,1,0,0.905946
4,2024-04-22 09:50:00,0,1,-0.0301,0.1287,0.036437,159041.0,497.43,497.60,497.29,...,0.199,-0.706,0.777,0.935,0.057,0,1,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,2024-05-03 15:35:00,1,0,0.0704,-0.0801,0.109146,108085.0,511.48,511.48,511.63,...,0.047,-1.106,0.954,2.506,8.660,1,0,0,0,0.000000
776,2024-05-03 15:40:00,0,1,-0.0137,0.0704,0.066151,148962.0,511.57,511.56,511.22,...,-0.976,-0.134,0.994,1.220,0.818,0,1,0,0,0.000000
777,2024-05-03 15:45:00,0,0,-0.0665,-0.0137,0.110382,90649.0,511.51,511.49,511.58,...,0.775,-0.221,0.948,1.551,2.396,0,0,1,0,0.575009
778,2024-05-03 15:50:00,1,0,0.0685,-0.0665,0.089017,187493.0,511.50,511.50,511.51,...,-0.690,-0.326,0.935,1.016,0.284,1,0,0,0,0.000000


In [31]:
df = df[~df['period'].str.contains('09:30:00')].dropna().set_index('period')
df.index = pd.to_datetime(df.index)
df

,movement,movement_prev_period,chg,chg_prev_period,vpoc_prev_period,total_volume_prev_period,vpoc_price_prev_period,tpoc_price_prev_period,open_prev_period,high_prev_period,...,price_skew_prev_period,price_kurt_prev_period,vol_price_cv_prev_period,vol_price_skew_prev_period,vol_price_kurt_prev_period,trapped_longs_prev_period,trapped_shorts_prev_period,trapped_shorts_at_extremes_prev_period,trapped_longs_at_extremes_prev_period,trapped_at_extremes_volume_prev_period
period,,,,,,,,,,,,,,,,,,,,,
2024-04-22 09:35:00,1,0,0.0080,-0.1265,0.247937,628143.0,497.88,497.74,497.84,498.07,...,-0.342,-0.787,2.370,8.560,78.235,1,0,0,0,0.000000
2024-04-22 09:40:00,1,1,0.0080,0.0080,0.085825,206035.0,497.23,497.24,497.20,497.33,...,-0.335,-0.872,0.881,1.170,1.209,0,1,0,0,0.000000
2024-04-22 09:45:00,1,1,0.1287,0.0080,0.075736,166394.0,497.11,497.00,497.24,497.34,...,-0.027,-0.635,1.006,1.646,2.898,0,1,1,0,0.905946
2024-04-22 09:50:00,0,1,-0.0301,0.1287,0.036437,159041.0,497.43,497.60,497.29,497.96,...,0.199,-0.706,0.777,0.935,0.057,0,1,0,0,0.000000
2024-04-22 09:55:00,0,0,-0.0442,-0.0301,0.082666,129267.0,498.00,497.86,497.94,498.01,...,0.304,-0.391,0.806,0.777,-0.466,1,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-03 15:35:00,1,0,0.0704,-0.0801,0.109146,108085.0,511.48,511.48,511.63,511.65,...,0.047,-1.106,0.954,2.506,8.660,1,0,0,0,0.000000
2024-05-03 15:40:00,0,1,-0.0137,0.0704,0.066151,148962.0,511.57,511.56,511.22,511.75,...,-0.976,-0.134,0.994,1.220,0.818,0,1,0,0,0.000000
2024-05-03 15:45:00,0,0,-0.0665,-0.0137,0.110382,90649.0,511.51,511.49,511.58,511.74,...,0.775,-0.221,0.948,1.551,2.396,0,0,1,0,0.575009


In [32]:
historical_spy_trades = pd.read_csv('historical_spy_trades.csv',index_col=0)
historical_spy_trades.index = pd.to_datetime(historical_spy_trades.index)
historical_spy_trades = historical_spy_trades.tz_localize(None)
historical_spy_trades

,price,volume
timestamp,,
2024-04-22 09:30:00,497.84,100.0
2024-04-22 09:30:00,497.88,145191.0
2024-04-22 09:30:00,497.85,1.0
2024-04-22 09:30:00,497.85,1.0
2024-04-22 09:30:00,497.85,2.0
...,...,...
2024-05-03 15:59:59,511.28,200.0
2024-05-03 15:59:59,511.28,200.0
2024-05-03 15:59:59,511.30,100.0


In [33]:
encoder = LabelEncoder()
df['period_idx'] = encoder.fit_transform(df.index.time)

In [34]:
tshorts = df[df.trapped_shorts_at_extremes_prev_period==1]
tlongs = df[df.trapped_longs_at_extremes_prev_period==1]
tshorts.shape, tlongs.shape

((156, 33), (156, 33))

In [35]:
days = df.index.date

for day in days:
    volume_at_930 = df.loc[str(day) + " 09:35:00", 'total_volume_prev_period']
    vpoc_at_930 = df.loc[str(day) + " 09:35:00", 'vpoc_prev_period']

    # relative_volume_to_930
    df.loc[str(day):str(day) + " ", 'relative_volume_to_930'] = df.loc[str(day):str(day) + " ", 'total_volume_prev_period'] / volume_at_930

    # relative_volume_to_prev
    df.loc[str(day):str(day) + " ", 'relative_volume_prev_period'] = df.loc[str(day):str(day) + " ", 'total_volume_prev_period'] / df.loc[str(day):str(day) + " ", 'total_volume_prev_period'].shift(1,fill_value=volume_at_930)

    # # relative_vpoc_prev_period
    # df.loc[str(day):str(day) + " ", 'relative_vpoc_prev_period'] = df.loc[str(day):str(day) + " ", 'vpoc_prev_period'] / df.loc[str(day):str(day) + " ", 'vpoc_prev_period'].shift(1,fill_value=vpoc_at_930)    

In [36]:
MAX_TRADE_DURATION = 15 #mins
TP_SHORT = 1 #dollars
SL_SHORT = 1 #dollars
TP_LONG = 1 #dollars
SL_LONG = 1 #dollars

# df = df.loc["2024-05-03"]

df['success'] = 0

for period in df.index:
    entry_zone = df.loc[period]
    trading_zone = historical_spy_trades.loc[period:period+dt.timedelta(minutes=MAX_TRADE_DURATION)]
    entry_price = trading_zone.price.values[0]

    if entry_zone.trapped_shorts_at_extremes_prev_period:
        
        stop_loss, stop_loss_period = (entry_price - trading_zone.price >= SL_LONG).any(), (entry_price - trading_zone.price >= SL_LONG).idxmax()
        take_profit, take_profit_period = (trading_zone.price - entry_price >= TP_LONG).any(), (trading_zone.price - entry_price >= TP_LONG).idxmax()
        
        if take_profit and not stop_loss:
            df.loc[period,'success'] = 1
        elif take_profit and stop_loss and take_profit_period < stop_loss_period:
            df.loc[period,'success'] = 1
        elif not take_profit and not stop_loss:
            df.loc[period,'success'] = max(0,np.sign(trading_zone.iloc[-1].price - entry_price))

    elif entry_zone.trapped_longs_at_extremes_prev_period:
        
        stop_loss, stop_loss_period = (trading_zone.price - entry_price >= SL_SHORT).any(), (trading_zone.price - entry_price >= SL_SHORT).idxmax()
        take_profit, take_profit_period = (entry_price - trading_zone.price >= TP_SHORT).any(), (entry_price - trading_zone.price >= TP_SHORT).idxmax()

        if take_profit and not stop_loss:
            df.loc[period,'success'] = 1
        elif take_profit and stop_loss and take_profit_period < stop_loss_period:
            df.loc[period,'success'] = 1
        elif not take_profit and not stop_loss:
            df.loc[period,'success'] = max(0,np.sign(entry_price - trading_zone.iloc[-1].price))

print(df[df.trapped_longs_at_extremes_prev_period==1].success.value_counts(), df[df.trapped_shorts_at_extremes_prev_period==1].success.value_counts())
print(df[df.trapped_longs_at_extremes_prev_period==1].success.value_counts(normalize=True), df[df.trapped_shorts_at_extremes_prev_period==1].success.value_counts(normalize=True))

success
0    88
1    68
Name: count, dtype: int64 success
1    92
0    64
Name: count, dtype: int64
success
0    0.564103
1    0.435897
Name: proportion, dtype: float64 success
1    0.589744
0    0.410256
Name: proportion, dtype: float64


In [37]:
df.to_csv('spy_engineered_v4.csv')